In [2]:
#01 IMPORT LIBRARIES

import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot

import requests
import logging
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import sqlite3

import xlwings as xw
from pathlib import Path


logging.getLogger().setLevel(logging.INFO)


c:\Users\Person\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#02 DATABASE CLASS

class DataBase():
    
    database_file = 'tradingview_database.db'
    
    @classmethod
    def AddToDatabase(cls, data): #method that innitiates the write-to-database method with the needed parameters
        """
        data = Instance of ScrapeTrendingView() Class
        """
        
        __class__.StoreInDatabase(data=data.income_statement, table_name=(data.exchange + '-' + data.company_ticker + '/Income-Statement'))
        __class__.StoreInDatabase(data=data.balanse_sheet, table_name=(data.exchange + '-' + data.company_ticker + '/Balance-Sheet'))
        __class__.StoreInDatabase(data=data.cashflow_statement, table_name=(data.exchange + '-' + data.company_ticker + '/Cashflow-Statement'))
        __class__.StoreInDatabase(data=data.statistics, table_name=(data.exchange + '-' + data.company_ticker + '/Ratios'))
        __class__.StoreInDatabase(data=data.company_data, table_name=(data.exchange + '-' + data.company_ticker + '/Company-Data'))

    @classmethod
    def StoreInDatabase(cls, data, table_name): #method that writes the data into the database
        """
        data = ScrapeTrendingView().income_statement \n
        data = ScrapeTrendingView().balanse_sheet \n
        data = ScrapeTrendingView().cashflow_statement \n
        data = ScrapeTrendingView().statistics \n
        data = ScrapeTrendingView().company_data \n\n

        table_name = Exchange-Ticker/Statement-Type \n
        Examples: \n
        table_name = NASDAQ-AAPL/Income-Statement \n
        table_name = NASDAQ-AAPL/Balance-Sheet \n
        table_name = NASDAQ-AAPL/Cashflow-Statement \n
        table_name = NASDAQ-AAPL/Ratios \n
        table_name = NASDAQ-AAPL/Company-Data \n
        """

        logging.info(f'Storring {table_name} in {__class__.database_file}')
        conn = sqlite3.connect(__class__.database_file) #connect to sqlite3-database
        data.to_sql(name=table_name, con=conn, if_exists='replace',  index=True) #store dataframe to sqlite-database
        conn.close()

    @classmethod
    def ReadFromDatabase(cls, table_name): #read data from Database, if available return the data if not return None
        """
        table_name = Exchange-Ticker/Statement-Type \n
        Examples: \n
        table_name = NASDAQ-AAPL/Income-Statement \n
        table_name = NASDAQ-AAPL/Balance-Sheet \n
        table_name = NASDAQ-AAPL/Cashflow-Statement \n
        table_name = NASDAQ-AAPL/Ratios \n
        table_name = NASDAQ-AAPL/Company-Data \n
        """
        conn = sqlite3.connect(__class__.database_file)

        try: #get the data from database and put it in pandas-dataframe
            output = pd.read_sql_query(f"SELECT * from '{table_name}'", conn, index_col='index') 
            logging.info(f"Load {table_name} from DataBase")
            
        except : #if data not available, return None
            logging.error(f'ERROR: {table_name}, not found in {__class__.database_file}')
            conn.close()
            return None

        conn.close()
        return output

    @classmethod
    def DropTable(cls, table_name_prefix): 
        """table_name=NASDAQ-AAPL"""

        table_name_suffix = ['/Income-Statement', '/Balance-Sheet', '/Cashflow-Statement', '/Ratios', '/Company-Data']
        #Connecting to sqlite
        conn = sqlite3.connect(__class__.database_file)

        #Creating a cursor object using the cursor() method
        cursor = conn.cursor()

        #Droping  table if already exists
        for i in range(0,len(table_name_suffix)):
            try:
                cursor.execute(f"DROP TABLE '{table_name_prefix + table_name_suffix[i]}'")
                logging.info(f"Table {table_name_prefix + table_name_suffix[i]} dropped... ")
            except sqlite3.OperationalError:
                logging.error(f' Error removing {table_name_prefix + table_name_suffix[i]}. Table is probably not availabe in the database!')

        # Commit your changes in the database
        conn.commit()

        #Closing the connection
        conn.close()


    def GetFromDataBase(self, company_url):
        #get financial data from DataBase
        self.income_statement = __class__.ReadFromDatabase(table_name= company_url + "/Income-Statement") #attempt to get the data from database
        self.balanse_sheet = __class__.ReadFromDatabase(table_name= company_url + "/Balance-Sheet") #attempt to get the data from database
        self.cashflow_statement = __class__.ReadFromDatabase(table_name= company_url + "/Cashflow-Statement") #attempt to get the data from database
        self.statistics = __class__.ReadFromDatabase(table_name= company_url + "/Ratios") #attempt to get the data from database
        self.company_data = __class__.ReadFromDatabase(table_name= company_url + "/Company-Data") #attempt to get the data from database
        
        if self.company_data is not None:
            self.company_name = self.company_data.loc['company_name'][0]
            self.company_ticker = self.company_data.loc['company_ticker'][0]
            
# DataBase.DropTable(table_name_prefix='NASDAQ-AAPL')
# DataBase.DropTable()


In [43]:
#EXCEL CLASS
class Excel():
    wb = None
    sht = None
    height = None
    width = None

    insert_row = None
    insert_col = None

    def start(name="Income Statement", height=375, width=550):
        # Create an empty workbook & rename sheet
        __class__.wb = xw.Book()
        __class__.sht = __class__.wb.sheets[0]
        __class__.sht.name = name

        __class__.height = height
        __class__.width = width

        __class__.insert_row = 1 #default insert cell-row
        __class__.insert_col = 'A' #default insert cell-column

        # Create output directory
        OUTPUT_DIR = Path.cwd() / 'Output'
        OUTPUT_DIR.mkdir(exist_ok=True)

    # Helper function to insert 'Headings' into Excel cells
    def insert_heading(cell, text):
        """ insert_heading(sht.range("A2"), "Matplotlib Chart") """
        rng = __class__.sht.range(cell)
        rng.value = text
        rng.font.bold = True
        rng.font.size = 24
        rng.font.color = (0, 0, 139)

    def insert_visual(fig, name):

        cell = __class__.insert_col + str(__class__.insert_row) #calculate the cell where the visual will be inserted
        logging.info(cell)

        __class__.sht.pictures.add(fig,
                                name = name,
                                update = True,
                                left = __class__.sht.range(cell).left,
                                top = __class__.sht.range(cell).top,
                                height = __class__.height,
                                width = __class__.width)

        __class__.insert_row = __class__.insert_row + 25 #increase row-number to offset the next visual
        logging.info(__class__.insert_row)
    
    def close():
        __class__.wb.save(__class__.OUTPUT_DIR / "PythonCharts.xlsx")
        if len(__class__.wb.app.books) == 1:
            __class__.wb.app.quit()
        else:
            __class__.wb.close()


In [5]:
#03 SCRAPE TRENDINGVIEW CLASS

class ScrapeTrendingView():

    maximum_number_of_colapsed_rows = 50

    def __init__(self, company_url):
        """
        company_url = Exchange-Ticker \n
        company_url = NASDAQ-AAPL \n
        """

        # innitialize and set chrome-webdriver options
        chrome_options = Options()
        chrome_options.add_argument("--start-maximized")
        # self.chrome_options.add_argument("--window-size=1000,1080")
        # chrome_options.add_argument("--headless")

        self.driver = webdriver.Chrome("G:\My Drive\Investing\Programming\chromedriver.exe", options=chrome_options)
        self.driver.implicitly_wait(2)
        self.time_sleep = 2
        # self.driver.maximize_window()

        self.scrapeIncomeStatement(company_url=company_url) #start Income-Statement scraping
        self.scrapeBalanceSheet(company_url=company_url) #start Balance-Sheet scraping
        self.scrapeCashFlow(company_url=company_url) #start Cashflow-Statement scraping
        self.scrapeStatistics(company_url=company_url) #start Ratios scraping
        self.scrapeCompanyData(company_url = company_url) #start Company-Data scraping

        #add additional data to dataframe
        self.companyData_to_dataframe()

        self.driver.close()

    def scrapeIncomeStatement(self, company_url):

        self.income_statement_url = "https://www.tradingview.com/symbols/" + \
            company_url + "/financials-income-statement/?selected="

        self.driver.get(self.income_statement_url)
        time.sleep(self.time_sleep)

        self.switch_annual_data()
        self.close_cookies_popup()
        self.wait_page_to_load()

        # expand income-statement collapsed rows
        i = 0
        while True:
            i = i+1
            if i > __class__.maximum_number_of_colapsed_rows:
                logging.error(f'Break While loop i={i}')
                break
            try:
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                break

        # scrape the data
        output = self.scrape_the_data()

        self.income_statement = self.scraped_data_to_dataframe(output=output)

    def scrapeBalanceSheet(self, company_url):
        logging.info('Start Balance Sheet Scrape')
        self.balanse_sheet_url = "https://www.tradingview.com/symbols/" + \
            company_url + "/financials-balance-sheet/?selected="
        self.driver.get(self.balanse_sheet_url)
        time.sleep(self.time_sleep)

        self.switch_annual_data()
        self.wait_page_to_load()
        # self.close_cookies_popup()

        # expand balance-sheet collapsed-rows level-1
        i = 0
        while True:
            # logging.info('Start Expanding Balance Sheet Rows Level-1')
            i = i+1
            if i > __class__.maximum_number_of_colapsed_rows:
                logging.error(f'Break While loop i={i}')
                break
            try:
                # expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k hasChanges-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding Balance Sheet Rows Level-1')
                break

         # expand balance-sheet collapsed-rows level-2
        i = 0
        while True:
            # logging.info('Start Expanding Balance Sheet Rows Level-2')
            i = i+1
            if i >  __class__.maximum_number_of_colapsed_rows:
                logging.error(f'Break While loop i={i}')
                break
            try:
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding Balance Sheet Rows Level-2')
                break

        # scrape the data
        output = self.scrape_the_data()
        self.balanse_sheet = self.scraped_data_to_dataframe(output=output)

    def scrapeCashFlow(self, company_url):

        logging.info('Start CashFlow Scrape')
        self.cashflow_url = "https://www.tradingview.com/symbols/" + \
            company_url + "/financials-cash-flow/?selected="

        self.driver.get(self.cashflow_url)
        time.sleep(self.time_sleep)

        self.switch_annual_data()
        self.wait_page_to_load()
        # self.close_cookies_popup()

        # expand cash-flow collapsed-rows level-1
        i = 0
        while True:
            # logging.info('Start Expanding CashFlow Rows Level-1')
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                # expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k hasChanges-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding CashFlow Rows Level-1')
                break

        # expand cash-flow collapsed-rows level-2
        i = 0
        while True:
            # logging.info('Start Expanding CashFlow Rows Level-2')
            i = i+1
            if i > 20:
                logging.info(f'Break While loop i={i}')
                break
            try:
                expand_arrow_xpath = "//span[@class='arrow-_PBNXQ7k']"
                expand_arrow_element = self.driver.find_element_by_xpath(
                    expand_arrow_xpath)
                # self.driver.execute_script("arguments[0].scrollIntoView();", expand_arrow_element) #scroll view to element

                expand_arrow_element.click()
                # print(expand_arrow_element.if_exists)

            except:
                # logging.info('End Expanding CashFlow Rows Level-2')
                break

        # scrape the data
        output = self.scrape_the_data()
        self.cashflow_statement = self.scraped_data_to_dataframe(output=output)

    def scrapeStatistics(self, company_url):
        logging.info('Start Statistics Scrape')
        self.statistics_url = "https://www.tradingview.com/symbols/" + company_url + "/financials-statistics-and-ratios/?selected="
        self.driver.get(self.statistics_url)
        time.sleep(self.time_sleep)

        # self.close_cookies_popup()
        self.switch_annual_data()
        self.wait_page_to_load()

        statistics_table_xpath = "//div[@class='container-YOfamMRP']/div"
        statistics_table_rows = self.driver.find_elements_by_xpath(
            statistics_table_xpath)

        # for item in financial_table[1]:
        #     print(item)

        # print(self.financial_table.text)
        # print(len(statistics_table_rows))
        output = []

        for item in statistics_table_rows:
            item_list = item.text.splitlines()
            output_temp = []

            # skip non-data items like Key stats, Profitability ratios, Liquidity ratios, Solvency ratios
            if len(item_list) == 1:
                continue
            else:
                for i in range(len(item_list)):

                    output_temp.append(item_list[i].replace(
                        '\u202a', '').replace('\u202c', ''))
                    # print(temp[i])

                # print(type(temp), len(temp))
                # print(temp)
                output.append(output_temp)

        # for item in output:
        #     print(len(item),item)
        #     pass

        self.statistics = self.scraped_data_to_dataframe(output=output)


    def scrapeCompanyData(self, company_url):
        self.company_data_url = "https://www.tradingview.com/symbols/" + company_url + "/forecast/"
        self.driver.get(self.company_data_url)
        time.sleep(self.time_sleep)
        self.wait_page_to_load()

        company_name_css = "h2[class='tv-symbol-header__first-line']"
        company_name_element = self.driver.find_element(By.CSS_SELECTOR, company_name_css)
        self.company_name = company_name_element.text
        # print(self.company_name)

        company_ticker_css = "span[class='tv-symbol-header__second-line tv-symbol-header__second-line--with-hover js-symbol-dropdown'] span[class='tv-symbol-header__second-line--text']"
        company_ticker_element = self.driver.find_element(By.CSS_SELECTOR, company_ticker_css)
        self.company_ticker = company_ticker_element.text
        # print(self.company_ticker)

        exchage_css = "span[class='tv-symbol-header__second-line tv-symbol-header__second-line--with-hover js-symbol-dropdown'] span[class='tv-symbol-header__exchange']" #NASDAQ/NYSE
        exchage_element = self.driver.find_element(By.CSS_SELECTOR, exchage_css)
        self.exchange = exchage_element.text

    def companyData_to_dataframe(self):

        data = {'income_statement_url': self.income_statement_url,
                'balanse_sheet_url': self.balanse_sheet_url,
                'cashflow_url': self.cashflow_url,
                'statistics_url': self.statistics_url,
                'company_data_url': self.company_data_url,
                'company_name':self.company_name,
                'company_ticker': self.company_ticker,
                "exchange": self.exchange}

        self.company_data = pd.DataFrame.from_dict(data, orient='index')
        
    
    def close_cookies_popup(self):
        cookie_button_xpath = "//button[@class='acceptAll-WvyPjcpY button-OvB35Th_ size-xsmall-OvB35Th_ color-brand-OvB35Th_ variant-primary-OvB35Th_']"
        cookie_button_element = self.driver.find_element_by_xpath(
            cookie_button_xpath)
        cookie_button_element.click()

    def switch_annual_data(self):
        annual_button_xpath = "//button[@id='FY']"
        annual_button_element = self.driver.find_element_by_xpath(
            annual_button_xpath)
        annual_button_element.click()

    def scraped_data_to_dataframe(self, output):
        output_index = []
        output_values = []
        output_colums = output[0][1:]
        self.currency = output[0][0].replace('Currency: ', '')
        # print(self.currency)

        for i in range(1, len(output)):
            output_index.append(output[i][0])
            output_values.append(output[i][1:])

        # apply neccessery correction to fix the values-data
        output_values = self.fix_data_values(input_data=output_values)
        df = pd.DataFrame(output_values, columns=output_colums,
                          index=output_index)  # add scraped data to dataframe
        return df

    def scrape_the_data(self):

        # financial_table_xpath = "//div[@class='container-YOfamMRP']"
        financial_table_xpath = "//div[@class='container-YOfamMRP']/div"
        financial_table_rows = self.driver.find_elements_by_xpath(
            financial_table_xpath)
        # print(len(financial_table_rows))

        output = []
        number_of_periods = len(financial_table_rows[0].text.splitlines())

        for item in financial_table_rows:
            item_list = item.text.splitlines()
            output_temp = []

            if len(item_list) == number_of_periods:  # rows without YOY-grow

                for i in range(len(item_list)):
                    output_temp.append(item_list[i].replace(
                        '\u202a', '').replace('\u202c', ''))
            else:  # rows with YOY-grow
                if 'YoY growth' in item_list:  # Quarterly
                    for i in range(0, len(item_list), 2):  # skip YOY-grow row
                        output_temp.append(item_list[i].replace(
                            '\u202a', '').replace('\u202c', ''))
                else:  # Anual report
                    output_temp.append(item_list[0])
                    for i in range(1, len(item_list), 2):  # skip YOY-grow row
                        output_temp.append(item_list[i].replace(
                            '\u202a', '').replace('\u202c', ''))

            output.append(output_temp)

        return output


    def fix_data_values(self, input_data):
        output = []

        for row in input_data:
            output_row = []
            for item in row:
                # print(f'item={item}')

                if '−' in item:  # convert minus sign to real minus, for some reason the sign is not recognized as minus
                    item = item.replace('−', '-')

                if 'T' in item:  # convert Trillion-values to numeric
                    item = item.replace('T', '')
                    item = float(item)
                    item = item*1000000000000
                    # item = int(item)

                elif 'B' in item:  # convert Billion-values to numeric
                    item = item.replace('B', '')
                    item = float(item)
                    item = item*1000000000
                    # item = int(item)

                elif 'M' in item:  # convert Milion-values to numeric
                    item = item.replace('M', '')
                    item = float(item)
                    item = item*1000000
                    # item = int(item)

                elif 'K' in item:  # convert Thousants-values to numeric
                    item = item.replace('K', '')
                    item = float(item)
                    item = item*1000
                    # item = int(item)

                if isinstance(item, str):  # if item is not integer (0.00, ---, -)

                    if '—' in item:  # set value to None
                        item = None

                    elif '.' in item:  # convert value to float
                        item = float(item)

                if self.currency != 'USD':  # convert values to USD

                    if self.currency == 'KRW':
                        self.multiplier = 0.000700680009950

                    # check if item is int or float
                    if isinstance(item, float) or isinstance(item, int):
                        item = item*self.multiplier

                output_row.append(item)

            output.append(output_row)

        return output

    def wait_page_to_load(self):
        """wait the page to be fully loaded before proceeding with the scraping procedures"""

        price_css = "div[class='tv-symbol-price-quote__value js-symbol-last']"
        price_element = self.driver.find_element(By.CSS_SELECTOR, price_css)

        currency_symbol_css = ".tv-symbol-price-quote__currency.js-symbol-currency"
        currency_symbol_element = self.driver.find_element(By.CSS_SELECTOR, currency_symbol_css)

        prince_increase_css = ".js-symbol-change.tv-symbol-price-quote__change-value"
        prince_increase_element = self.driver.find_element(By.CSS_SELECTOR, prince_increase_css)

        prince_increase_percent_css = ".js-symbol-change-pt.tv-symbol-price-quote__change-value"
        prince_increase_percent_element = self.driver.find_element(By.CSS_SELECTOR, prince_increase_percent_css)

    

In [6]:
#04 HELPER CLASS

class Helper():

    def generateClassVariables(self, company_statement):
        """ Generate Income Statement / Balance Sheet / Cashflow Statement / Statistics variables """

        self.items = company_statement.transpose()
        # self.item_keys = []

        for item in self.items:
            item_temp = item
            item_temp = item_temp.lower()
            item_temp = item_temp.replace(' - ', '_')
            item_temp = item_temp.replace(' – ', '_')
            item_temp = item_temp.replace('-', '_')
            item_temp = item_temp.replace(' ', '_')
            item_temp = item_temp.replace("'", '')
            item_temp = item_temp.replace('"', '')
            item_temp = item_temp.replace('(', '')
            item_temp = item_temp.replace(')', '')
            item_temp = item_temp.replace('&', 'and')
            item_temp = item_temp.replace('.', '')
            item_temp = item_temp.replace(',', '')
            item_temp = item_temp.replace('/', '_')
            item_temp = item_temp.replace('%', 'percent')
            print(f'self.{item_temp}_str = "{item}"')
            # self.item_keys.append(item_temp)


        # self.res = {}
        # for key in self.item_keys:
        #     for value in self.items:
        #         self.res[key] = value
        #         self.items.remove(value)
        #         break


        # print(self.res)

# helper = Helper()
# helper.generateClassVariables(companies[0].statistics)

In [7]:
#05 INCOME STATEMENT VISUALIZER CLASS

class IncomeStatementVisualizer():
    
    def __init__(self, company_data):

        self.company_name = company_data.company_data.loc['company_name'][0] #get company_name from the dataframe
        self.df_income_statement = company_data.income_statement.transpose()
        self.income_statement_vars()
    
    def income_statement_vars(self):

        self.total_revenue_str = "Total revenue"
        self.cost_of_goods_sold_str = "Cost of goods sold"
        self.deprecation_and_amortization_str = "Deprecation and amortization"
        self.depreciation_str = "Depreciation"
        self.amortization_of_intangibles_str = "Amortization of intangibles"
        self.amortization_of_deferred_charges_str = "Amortization of deferred charges"
        self.other_cost_of_goods_sold_str = "Other cost of goods sold"
        self.gross_profit_str = "Gross profit"
        self.operating_expenses_excl_cogs_str = "Operating expenses (excl. COGS)"
        self.selling_general_admin_expenses_total_str = "Selling/general/admin expenses, total"
        self.research_and_development_str = "Research & development"
        self.selling_general_admin_expenses_other_str = "Selling/general/admin expenses, other"
        self.other_operating_expenses_total_str = "Other operating expenses, total"
        self.operating_income_str = "Operating income"
        self.non_operating_income_total_str = "Non-operating income, total"
        self.interest_expense_net_of_interest_capitalized_str = "Interest expense, net of interest capitalized"
        self.interest_expense_on_debt_str = "Interest expense on debt"
        self.interest_capitalized_str = "Interest capitalized"
        self.non_operating_income_excl_interest_expenses_str = "Non-operating income, excl. interest expenses"
        self.non_operating_interest_income_str = "Non-operating interest income"
        self.pretax_equity_in_earnings_str = "Pretax equity in earnings"
        self.miscellaneous_non_operating_expense_str = "Miscellaneous non-operating expense"
        self.unusual_income_expense_str = "Unusual income/expense"
        self.impairments_str = "Impairments"
        self.restructuring_charge_str = "Restructuring charge"
        self.legal_claim_expense_str = "Legal claim expense"
        self.unrealized_gain_loss_str = "Unrealized gain/loss"
        self.other_exceptional_charges_str = "Other exceptional charges"
        self.pretax_income_str = "Pretax income"
        self.equity_in_earnings_str = "Equity in earnings"
        self.taxes_str = "Taxes"
        self.income_tax_current_str = "Income tax, current"
        self.income_tax_current_domestic_str = "Income tax, current - domestic"
        self.income_tax_current_foreign_str = "Income Tax, current - foreign"
        self.income_tax_deferred_str = "Income tax, deferred"
        self.income_tax_deferred_domestic_str = "Income tax, deferred - domestic"
        self.income_tax_deferred_foreign_str = "Income tax, deferred - foreign"
        self.income_tax_credits_str = "Income Tax Credits"
        self.non_controlling_minority_interest_str = "Non-controlling/minority interest"
        self.after_tax_other_income_expense_str = "After tax other income/expense"
        self.net_income_before_discontinued_operations_str = "Net income before discontinued operations"
        self.discontinued_operations_str = "Discontinued operations"
        self.net_income_str = "Net income"
        self.dilution_adjustment_str = "Dilution adjustment"
        self.preferred_dividends_str = "Preferred dividends"
        self.diluted_net_income_available_to_common_stockholders_str = "Diluted net income available to common stockholders"
        self.basic_earnings_per_share_basic_eps_str = "Basic earnings per share (Basic EPS)"
        self.diluted_earnings_per_share_diluted_eps_str = "Diluted earnings per share (Diluted EPS)"
        self.average_basic_shares_outstanding_str = "Average basic shares outstanding"
        self.diluted_shares_outstanding_str = "Diluted shares outstanding"
        self.ebitda_str = "EBITDA"
        self.ebit_str = "EBIT"
        self.total_operating_expenses_str = "Total operating expenses"

    def show_all_visuals(self):
        self.revenue()
        self.operating_income()
        self.pretax_income()
        self.discontinued_operations()
        self.net_income()
        self.diluted_net_income()
        self.eps()
        self.shares()
        self.ebit()
        self.operating_expenses()

    def revenue(self):
        params = ['Total revenue', 'Cost of goods sold', 'Gross profit']
        title = 'Total Revenue - Cost Of Goods Sold = Gross Profit'
        self.graph_template(y_axis_data=params, graph_title=title)

    def operating_income(self):
        params = ['Operating expenses (excl. COGS)', 'Operating income', 'Gross profit']
        title = 'Gross Profit - Operating Expenses = Operating Income'
        self.graph_template(y_axis_data=params, graph_title=title)

    def pretax_income(self):
        params = ['Operating income', 'Non-operating income, total', 'Pretax income']
        title = 'Operating Income + Non Operating Income = Pretax Income'
        self.graph_template(y_axis_data=params, graph_title=title)

    def discontinued_operations(self):
        params = ['Non-controlling/minority interest', 'After tax other income/expense', 'Net income before discontinued operations', 'Discontinued operations']
        title = 'Net income before discontinued operations'
        self.graph_template(y_axis_data=params, graph_title=title)

    def net_income(self):
        params = ['Pretax income', 'Taxes', 'Net income']
        title = 'Pretax Income - Taxes = Net Income'
        self.graph_template(y_axis_data=params, graph_title=title)

    def diluted_net_income(self):
        params = ['Dilution adjustment', 'Preferred dividends', 'Diluted net income available to common stockholders']
        title = 'Net Income + Dilution Adjustment - Preferred Dividents = Diluted Net Income'
        self.graph_template(y_axis_data=params, graph_title=title)

    def eps(self):
        params = ['Basic earnings per share (Basic EPS)', 'Diluted earnings per share (Diluted EPS)']
        title = 'Earnings Per Share'
        self.graph_template(y_axis_data=params, graph_title=title)

    def shares(self):
        params = ['Average basic shares outstanding', 'Diluted shares outstanding']
        title = 'Shares Outstanding'
        self.graph_template(y_axis_data=params, graph_title=title)

    def ebit(self):
        params = ['EBITDA', 'EBIT']
        title = 'EBIT/EBITDA'
        self.graph_template(y_axis_data=params, graph_title=title)

    def operating_expenses(self):
        params = ['Operating expenses (excl. COGS)', 'Cost of goods sold', 'Total operating expenses']
        title = 'Operating Expenses + Cost of Goods Sold = Total Operating Expenses'
        self.graph_template(y_axis_data=params, graph_title=title)

    def graph_template(self, y_axis_data, graph_title):
        """
        y_axis_list=['Total revenue', 'Cost of goods sold', 'Gross profit'] 
        graph_title= f'{self.company_name} | Total Revenue - Cost Of Goods Sold = Gross Profit'
        """

        fig = px.line(self.df_income_statement,
                        x = self.df_income_statement.index, 
                        y = y_axis_data,
                        title = f'{self.company_name} | {graph_title}',
                        markers = True)
        fig.show()

    """ 
        OLD GRAPH FUNCTIONS
        def revenue(self):
            fig_revenue = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                y=['Total revenue', 'Cost of goods sold', 'Gross profit'],
                                title= f'{self.company_name} | Total Revenue - Cost Of Goods Sold = Gross Profit', markers=True)
            fig_revenue.show()

        def operating_income(self):
            fig_operating_income = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                            y=['Operating expenses (excl. COGS)', 'Operating income', 'Gross profit'],
                                            title='Gross Profit - Operating Expenses = Operating Income', markers=True)
            fig_operating_income.show()

        def pretax_income(self):
            fig_pretax_income = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                        y=['Operating income', 'Non-operating income, total', 'Pretax income'],
                                        title='Operating Income + Non Operating Income = Pretax Income', markers=True)
            fig_pretax_income.show()

        def discontinued_operations(self):
            fig_discontinued_operations = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                                    y=['Non-controlling/minority interest', 'After tax other income/expense',
                                                    'Net income before discontinued operations', 'Discontinued operations', ],
                                                    title='Net income before discontinued operations', markers=True)
            fig_discontinued_operations.show()

        def net_income(self):
            fig_net_income = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                    y=['Pretax income', 'Taxes', 'Net income'],
                                    title='Pretax Income - Taxes = Net Income', markers=True)
            fig_net_income.show()

        def diluted_net_income(self):
            fig_diluted_net_income = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                            y=['Dilution adjustment', 'Preferred dividends', 'Diluted net income available to common stockholders'],
                                            title='Net Income + Dilution Adjustment - Preferred Dividents = Diluted Net Income', markers=True)
            fig_diluted_net_income.show()

        def eps(self):
            fig_eps = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                y=['Basic earnings per share (Basic EPS)', 'Diluted earnings per share (Diluted EPS)'],
                                title='Earnings Per Share', markers=True)
            fig_eps.show()

        def shares(self):
            fig_shares = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                y=['Average basic shares outstanding', 'Diluted shares outstanding'],
                                title='Shares Outstanding', markers=True)
            fig_shares.show()

        def ebit(self):
            fig_ebit = px.line(self.df_income_statement, x=self.df_income_statement.index, 
                                y=['EBITDA', 'EBIT'],
                                title='EBIT/EBITDA', markers=True)
            fig_ebit.show()

    def operating_expenses(self):
    """

In [8]:
#06 BALANCE SHEET VISUALIZER CLASS

class BalanceSheetVisualizer():

    def __init__(self, company_data):
        
        self.company_name = company_data.company_data.loc['company_name'][0] #get company_name from the dataframe
        self.df_balanse_sheet = company_data.balanse_sheet.transpose()
        self.balanse_sheet_vars()
    
    def balanse_sheet_vars(self):

        self.total_assets_str = "Total assets"
        self.total_current_assets_str = "Total current assets"
        self.cash_and_short_term_investments_str = "Cash and short term investments"
        self.cash_and_equivalents_str = "Cash & equivalents"
        self.short_term_investments_str = "Short term investments"
        self.total_receivables_net_str = "Total receivables, net"
        self.accounts_receivable_trade_net_str = "Accounts receivable - trade, net"
        self.accounts_receivables_gross_str = "Accounts receivables, gross"
        self.bad_debt___doubtful_accounts_str = "Bad debt / Doubtful accounts"
        self.other_receivables_str = "Other receivables"
        self.total_inventory_str = "Total inventory"
        self.inventories_work_in_progress_str = "Inventories - work in progress"
        self.inventories_progress_payments_and_other_str = "Inventories - progress payments & other"
        self.inventories_finished_goods_str = "Inventories - finished goods"
        self.inventories_raw_materials_str = "Inventories - raw materials"
        self.prepaid_expenses_str = "Prepaid expenses"
        self.other_current_assets_total_str = "Other current assets, total"
        self.total_non_current_assets_str = "Total non-current assets"
        self.long_term_investments_str = "Long term investments"
        self.note_receivable_long_term_str = "Note receivable - long term"
        self.investments_in_unconsolidated_subsidiaries_str = "Investments in unconsolidated subsidiaries"
        self.other_investments_str = "Other investments"
        self.net_property_plant_equipment_str = "Net property/plant/equipment"
        self.gross_property_plant_equipment_str = "Gross property/plant/equipment"
        self.property_plant_equipment_buildings_str = "Property/plant/equipment - Buildings"
        self.property_plant_equipment_construction_in_progress_str = "Property/plant/equipment - Construction in progress"
        self.property_plant_equipment_machinery_and_equipment_str = "Property/plant/equipment - Machinery & equipment"
        self.property_plant_equipment_land_and_improvement_str = "Property/plant/equipment - Land & improvement"
        self.property_plant_equipment_leased_property_str = "Property/plant/equipment - Leased property"
        self.property_plant_equipment_leases_str = "Property/plant/equipment - Leases"
        self.property_plant_equipment_computer_software_and_equipment_str = "Property/plant/equipment - Computer software and equipment"
        self.property_plant_equipment_transportation_equipment_str = "Property/plant/equipment - Transportation equipment"
        self.property_plant_equipment_other_str = "Property/plant/equipment - Other"
        self.accumulated_depreciation_total_str = "Accumulated depreciation, total"
        self.accumulated_depreciation_buildings_str = "Accumulated depreciation - Buildings"
        self.accumulated_depreciation_construction_in_progress_str = "Accumulated depreciation - Construction in progress"
        self.accumulated_depreciation_machinery_and_equipment_str = "Accumulated depreciation - Machinery & equipment"
        self.accumulated_depreciation_land_and_improvement_str = "Accumulated depreciation - Land & improvement"
        self.accumulated_depreciation_leased_property_str = "Accumulated depreciation - Leased property"
        self.accumulated_depreciation_leases_str = "Accumulated depreciation - Leases"
        self.accumulated_depreciation_computer_software_and_equipment_str = "Accumulated depreciation - Computer software and equipment"
        self.accumulated_depreciation_transportation_equipment_str = "Accumulated depreciation - Transportation equipment"
        self.accumulated_depreciation_other_str = "Accumulated depreciation - Other"
        self.deferred_tax_assets_str = "Deferred tax assets"
        self.net_intangible_assets_str = "Net intangible assets"
        self.goodwill_net_str = "Goodwill, net"
        self.goodwill_gross_str = "Goodwill, gross"
        self.accumulated_goodwill_amortization_str = "Accumulated goodwill amortization"
        self.other_intangibles_net_str = "Other intangibles, net"
        self.other_intangibles_gross_str = "Other intangibles, gross"
        self.accumulated_amortization_of_other_intangibles_str = "Accumulated amortization of other intangibles"
        self.deferred_charges_str = "Deferred charges"
        self.other_long_term_assets_total_str = "Other long term assets, total"
        self.total_liabilities_str = "Total liabilities"
        self.total_current_liabilities_str = "Total current liabilities"
        self.short_term_debt_str = "Short term debt"
        self.current_portion_of_lt_debt_and_capital_leases_str = "Current portion of LT debt and capital leases"
        self.short_term_debt_excl_current_portion_of_lt_debt_str = "Short term debt excl. current portion of LT debt"
        self.notes_payable_str = "Notes payable"
        self.other_short_term_debt_str = "Other short term debt"
        self.accounts_payable_str = "Accounts payable"
        self.income_tax_payable_str = "Income tax payable"
        self.dividends_payable_str = "Dividends payable"
        self.accrued_payroll_str = "Accrued payroll"
        self.deferred_income_current_str = "Deferred income, current"
        self.other_current_liabilities_str = "Other current liabilities"
        self.total_non_current_liabilities_str = "Total non-current liabilities"
        self.long_term_debt_str = "Long term debt"
        self.long_term_debt_excl_lease_liabilities_str = "Long term debt excl. lease liabilities"
        self.capital_and_operating_lease_obligations_str = "Capital and operating lease obligations"
        self.capitalized_lease_obligations_str = "Capitalized lease obligations"
        self.operating_lease_liabilities_str = "Operating lease liabilities"
        self.provision_for_risks_and_charge_str = "Provision for risks & charge"
        self.deferred_tax_liabilities_str = "Deferred tax liabilities"
        self.deferred_income_non_current_str = "Deferred income, non-current"
        self.other_non_current_liabilities_total_str = "Other non-current liabilities, total"
        self.total_equity_str = "Total equity"
        self.shareholders_equity_str = "Shareholders' equity"
        self.common_equity_total_str = "Common equity, total"
        self.retained_earnings_str = "Retained earnings"
        self.paid_in_capital_str = "Paid in capital"
        self.common_stock_par_carrying_value_str = "Common stock par/Carrying value"
        self.additional_paid_in_capital_capital_surplus_str = "Additional paid-in capital/Capital surplus"
        self.treasury_stock_common_str = "Treasury stock - common"
        self.other_common_equity_str = "Other common equity"
        self.preferred_stock_carrying_value_str = "Preferred stock, carrying value"
        self.minority_interest_str = "Minority interest"
        self.total_liabilities_and_shareholders_equities_str = "Total liabilities & shareholders' equities"
        self.total_debt_str = "Total debt"
        self.net_debt_str = "Net debt"
        self.book_value_per_share_str = "Book value per share"

    def show_all_visuals(self):
        self.total_assets_liabilities_equity()
        self.current_non_current_assets()
        self.current_non_current_liabilities()
        self.total_debt_net_debt()
        self.book_value_per_share()


    def total_assets_liabilities_equity(self):
        params = ['Total assets', 'Total liabilities', 'Total equity',  "Total liabilities & shareholders' equities"]
        title = 'Total Assets/Liabilities/Equity'
        self.graph_template(y_axis_data=params, graph_title=title)
        

    def current_non_current_assets(self):
        params = ['Total current assets', 'Total non-current assets']
        title = 'Total Current/Non-Current Assets'
        self.graph_template(y_axis_data=params, graph_title=title)

    def current_non_current_liabilities(self):    
        params = ['Total current liabilities', 'Total non-current liabilities']
        title = 'Total Current/Non-Current Liabilities'
        self.graph_template(y_axis_data=params, graph_title=title)

    def total_debt_net_debt(self):    
        params = ['Total debt', 'Net debt']
        title = 'Total/Net Debt'
        self.graph_template(y_axis_data=params, graph_title=title)

    def book_value_per_share(self):    
        params = ['Book value per share']
        title = 'Book Value Per Share'
        self.graph_template(y_axis_data=params, graph_title=title)
    
    def graph_template(self, y_axis_data, graph_title):
        """
        y_axis_list=['Total revenue', 'Cost of goods sold', 'Gross profit'] 
        graph_title= f'{self.company_name} | Total Revenue - Cost Of Goods Sold = Gross Profit'
        """
        fig = px.line(self.df_balanse_sheet,
                        x = self.df_balanse_sheet.index, 
                        y = y_axis_data,
                        title = f'{self.company_name} | {graph_title}',
                        markers = True)
        fig.show()


    """ 
        OLD DEFs
        def total_assets_liabilities_equity(self):
                fig_total_assets_liabilities_equity = px.line(self.df_balanse_sheet, x=self.df_balanse_sheet.index, y=['Total assets', 'Total liabilities', 'Total equity',  "Total liabilities & shareholders' equities"],
                                                            title='Total Assets/Liabilities/Equity', markers=True)
                fig_total_assets_liabilities_equity.show()
                

        def current_non_current_assets(self):
            fig_current_non_current_assets = px.line(self.df_balanse_sheet, x=self.df_balanse_sheet.index, y=['Total current assets', 'Total non-current assets'],
                                                    title='Total Current/Non-Current Assets', markers=True)
            fig_current_non_current_assets.show()

        def current_non_current_liabilities(self):    
            fig_current_non_current_liabilities = px.line(self.df_balanse_sheet, x=self.df_balanse_sheet.index, y=['Total current liabilities', 'Total non-current liabilities'],
                                                        title='Total Current/Non-Current Liabilities', markers=True)
            fig_current_non_current_liabilities.show()

        def total_assets_liabilities_equity(self):    
            fig_total_assets_liabilities_equity = px.line(self.df_balanse_sheet, x=self.df_balanse_sheet.index, y=['Total debt', 'Net debt'],
                                                        title='Total/Net Debt', markers=True)
            fig_total_assets_liabilities_equity.show()

        def book_value_per_share(self):    
            fig_book_value_per_share = px.line(self.df_balanse_sheet, x=self.df_balanse_sheet.index, y=['Book value per share'],
                                            title='Book Value Per Share', markers=True)
            fig_book_value_per_share.show()
    """

In [9]:
#07 CASHFLOW STATEMENT VIZUALIZER CLASS

class CashflowStatementVisualizer():

    def __init__(self, company_data):

        self.company_name = company_data.company_data.loc['company_name'][0] #get company_name from the dataframe
        self.df_cashflow_statement = company_data.cashflow_statement.transpose()
        self.cashflow_statement_vars()
    
    def cashflow_statement_vars(self):

        self.cash_from_operating_activities_str = "Cash from operating activities"
        self.funds_from_operations_str = "Funds from operations"
        self.net_income_cash_flow_str = "Net income (cash flow)"
        self.depreciation_and_amortization_cash_flow_str = "Depreciation & amortization (cash flow)"
        self.depreciation_depletion_str = "Depreciation/depletion"
        self.amortization_str = "Amortization"
        self.deferred_taxes_cash_flow_str = "Deferred taxes (cash flow)"
        self.non_cash_items_str = "Non-cash items"
        self.changes_in_working_capital_str = "Changes in working capital"
        self.change_in_accounts_receivable_str = "Change in accounts receivable"
        self.change_in_taxes_payable_str = "Change in taxes payable"
        self.change_in_accounts_payable_str = "Change in accounts payable"
        self.change_in_accrued_expenses_str = "Change in accrued expenses"
        self.change_in_inventories_str = "Change in inventories"
        self.change_in_other_assets_liabilities_str = "Change in other assets/liabilities"
        self.cash_from_investing_activities_str = "Cash from investing activities"
        self.purchase_sale_of_business_net_str = "Purchase/sale of business, net"
        self.sale_of_fixed_assets_and_businesses_str = "Sale of fixed assets & businesses"
        self.purchase_acquisition_of_business_str = "Purchase/acquisition of business"
        self.purchase_sale_of_investments_net_str = "Purchase/sale of investments, net"
        self.sale_maturity_of_investments_str = "Sale/maturity of investments"
        self.purchase_of_investments_str = "Purchase of investments"
        self.capital_expenditures_str = "Capital expenditures"
        self.capital_expenditures_fixed_assets_str = "Capital expenditures - fixed assets"
        self.capital_expenditures_other_assets_str = "Capital expenditures - other assets"
        self.other_investing_cash_flow_items_total_str = "Other investing cash flow items, total"
        self.investing_activities_other_sources_str = "Investing activities – other sources"
        self.investing_activities_other_uses_str = "Investing activities – other uses"
        self.cash_from_financing_activities_str = "Cash from financing activities"
        self.issuance_retirement_of_stock_net_str = "Issuance/retirement of stock, net"
        self.sale_of_common_and_preferred_stock_str = "Sale of common & preferred stock"
        self.repurchase_of_common_and_preferred_stock_str = "Repurchase of common & preferred stock"
        self.issuance_retirement_of_debt_net_str = "Issuance/retirement of debt, net"
        self.issuance_retirement_of_long_term_debt_str = "Issuance/retirement of long term debt"
        self.issuance_of_long_term_debt_str = "Issuance of long term debt"
        self.reduction_of_long_term_debt_str = "Reduction of long term debt"
        self.issuance_retirement_of_short_term_debt_str = "Issuance/retirement of short term debt"
        self.issuance_retirement_of_other_debt_str = "Issuance/retirement of other debt"
        self.total_cash_dividends_paid_str = "Total cash dividends paid"
        self.common_dividends_paid_str = "Common dividends paid"
        self.preferred_dividends_paid_str = "Preferred dividends paid"
        self.other_financing_cash_flow_items_total_str = "Other financing cash flow items, total"
        self.financing_activities_other_sources_str = "Financing activities – other sources"
        self.financing_activities_other_uses_str = "Financing activities – other uses"
        self.free_cash_flow_str = "Free cash flow"

    def show_all_visuals(self):
        self.cashflow_operating_investing_financial()
        self.cashflow_operating_activities()
        self.cashflow_investing_activities()
        self.cash_from_financing_activities()


    def cashflow_operating_investing_financial (self):
        # params = ['Cash from operating activities', 'Cash from investing activities','Cash from financing activities', 'Free cash flow']
        params = [self.cash_from_operating_activities_str,
                self.cash_from_investing_activities_str,
                self.cash_from_financing_activities_str,
                self.free_cash_flow_str]
        title = 'Cashflow From Operating/Investing/Financial Activities'
        self.graph_template(y_axis_data=params, graph_title=title)
        

    def cashflow_operating_activities(self):
        # params = ['Cash from operating activities', 'Funds from operations','Changes in working capital']
        params = [self.cash_from_operating_activities_str,
                self.funds_from_operations_str,
                self.changes_in_working_capital_str]
        title = 'Cash from operating activities + Funds from operations = Cashflow From Operating Activities'
        self.graph_template(y_axis_data=params, graph_title=title)

    def cashflow_investing_activities (self):
        # params = ['Cash from investing activities', 'Purchase/sale of business, net','Purchase/sale of investments, net', 'Capital expenditures', 'Other investing cash flow items, total']
        params = [ self.cash_from_investing_activities_str,
                self.purchase_sale_of_business_net_str,
                self.purchase_sale_of_investments_net_str,
                self.capital_expenditures_str,
                self.other_investing_cash_flow_items_total_str,]
        title = 'Purchase/sale of business + Purchase/sale of investments + Capital expenditures + Other investing cash flow items = Cash from investing activities'
        self.graph_template(y_axis_data=params, graph_title=title)


    def cash_from_financing_activities(self):
        params = [self.issuance_retirement_of_stock_net_str,
                    self.issuance_retirement_of_debt_net_str,
                    self.total_cash_dividends_paid_str,
                    self.other_financing_cash_flow_items_total_str]
        title =  '+'.join(params) + '= Cash from financial activities'
        self.graph_template(y_axis_data=params, graph_title=title)


    def graph_template(self, y_axis_data, graph_title):
            """
            y_axis_list=['Total revenue', 'Cost of goods sold', 'Gross profit'] 
            graph_title= f'{self.company_name} | Total Revenue - Cost Of Goods Sold = Gross Profit'
            """
            fig = px.line(self.df_cashflow_statement,
                            x = self.df_cashflow_statement.index, 
                            y = y_axis_data,
                            title = f'{self.company_name} | {graph_title}',
                            markers = True)

            fig.update_traces(
                mode="markers+lines", hovertemplate=None)
            fig.update_layout(
                hovermode="x", hoverlabel_namelength=-1, font=dict(size=10))
            fig.show()

    

""" 
    OLD DEFs
    def cashflow_operating_investing_financial (self):
        fig_cashflow_operating_investing_financial = px.line(self.df_cashflow_statement, x=self.df_cashflow_statement.index, y=['Cash from operating activities', 'Cash from investing activities',
                                                                                                'Cash from financing activities', 'Free cash flow'],
                                                            title='Cashflow From Operating/Investing/Financial Activities', markers=True)

        fig_cashflow_operating_investing_financial.update_traces(
            mode="markers+lines", hovertemplate=None)
        fig_cashflow_operating_investing_financial.update_layout(
            hovermode="x", hoverlabel_namelength=-1)
        fig_cashflow_operating_investing_financial.show()
        

    def cashflow_operating_activities(self):
        fig_cashflow_operating_activities = px.line(self.df_cashflow_statement, x=self.df_cashflow_statement.index, y=['Cash from operating activities', 'Funds from operations',
                                                                                        'Changes in working capital'],
                                                    title='Cash from operating activities + Funds from operations = Cashflow From Operating Activities', markers=True)
        fig_cashflow_operating_activities.show()

    def cashflow_investing_activities (self):
        fig_cashflow_investing_activities = px.line(self.df_cashflow_statement, x=self.df_cashflow_statement.index, y=['Cash from investing activities', 'Purchase/sale of business, net',
                                                                                        'Purchase/sale of investments, net', 'Capital expenditures', 'Other investing cash flow items, total'],
                                                    title='Purchase/sale of business + Purchase/sale of investments + Capital expenditures + Other investing cash flow items = Cash from investing activities', markers=True)
        fig_cashflow_investing_activities.show()

    def graph_template(self, y_axis_data, graph_title):
            
            #y_axis_list=['Total revenue', 'Cost of goods sold', 'Gross profit'] 
            #graph_title= f'{self.company_name} | Total Revenue - Cost Of Goods Sold = Gross Profit'
            
            fig = px.line(self.self.df_cashflow_statement,
                            x = self.self.df_cashflow_statement.index, 
                            y = y_axis_data,
                            title = f'{self.company_name} | {graph_title}',
                            markers = True)

            fig.update_traces(
                mode="markers+lines", hovertemplate=None)
            fig.update_layout(
                hovermode="x", hoverlabel_namelength=-1)
            fig.show()
    """


' \n    OLD DEFs\n    def cashflow_operating_investing_financial (self):\n        fig_cashflow_operating_investing_financial = px.line(self.df_cashflow_statement, x=self.df_cashflow_statement.index, y=[\'Cash from operating activities\', \'Cash from investing activities\',\n                                                                                                \'Cash from financing activities\', \'Free cash flow\'],\n                                                            title=\'Cashflow From Operating/Investing/Financial Activities\', markers=True)\n\n        fig_cashflow_operating_investing_financial.update_traces(\n            mode="markers+lines", hovertemplate=None)\n        fig_cashflow_operating_investing_financial.update_layout(\n            hovermode="x", hoverlabel_namelength=-1)\n        fig_cashflow_operating_investing_financial.show()\n        \n\n    def cashflow_operating_activities(self):\n        fig_cashflow_operating_activities = px.line(self.df_cashflow

In [10]:
#07 STATISTICS/RATIOS VIZUALIZER CLASS

class StatisticsRatiosVisualizer():

    def __init__(self, company_data):
        self.company_name = company_data.company_data.loc['company_name'][0] #get company_name from the dataframe
        self.df_statistics = company_data.statistics.transpose()
        self.statistics_ratios_vars()

    def statistics_ratios_vars(self):
        self.total_common_shares_outstanding_str = "Total common shares outstanding"
        self.float_shares_outstanding_str = "Float shares outstanding"
        self.number_of_employees_str = "Number of employees"
        self.number_of_shareholders_str = "Number of shareholders"
        self.price_to_earnings_ratio_str = "Price to earnings ratio"
        self.price_to_sales_ratio_str = "Price to sales ratio"
        self.price_to_cash_flow_ratio_str = "Price to cash flow ratio"
        self.price_to_book_ratio_str = "Price to book ratio"
        self.enterprise_value_str = "Enterprise value"
        self.enterprise_value_to_ebitda_ratio_str = "Enterprise value to EBITDA ratio"
        self.return_on_assets_percent_str = "Return on assets %"
        self.return_on_equity_percent_str = "Return on equity %"
        self.return_on_invested_capital_percent_str = "Return on invested capital %"
        self.gross_margin_percent_str = "Gross margin %"
        self.operating_margin_percent_str = "Operating margin %"
        self.ebitda_margin_percent_str = "EBITDA margin %"
        self.net_margin_percent_str = "Net margin %"
        self.quick_ratio_str = "Quick ratio"
        self.current_ratio_str = "Current ratio"
        self.inventory_turnover_str = "Inventory turnover"
        self.asset_turnover_str = "Asset turnover"
        self.debt_to_assets_ratio_str = "Debt to assets ratio"
        self.debt_to_equity_ratio_str = "Debt to equity ratio"
        self.long_term_debt_to_total_assets_ratio_str = "Long term debt to total assets ratio"

    def show_all_visuals(self):
        self.shares_outstanding()
        self.enterprice_values()
        self.numer_of_employees_shareholders()
        self.price_ratios()
        self.return_ratios()
        self.margins()
        self.dept_ratios()
        self.liquidity_ratios()

    def shares_outstanding(self):
        params = ['Total common shares outstanding', 'Float shares outstanding']
        title = 'Number of Shares'
        self.graph_template(y_axis_data=params, graph_title=title)
    
    def enterprice_values(self):
        params = ['Enterprise value']
        title = 'Enterprise value'
        self.graph_template(y_axis_data=params, graph_title=title)
    
    def numer_of_employees_shareholders(self):
        params = ['Number of employees', 'Number of shareholders']
        title = 'Number of employees/shareholders'
        self.graph_template(y_axis_data=params, graph_title=title)

    def price_ratios(self):
        params = ['Price to earnings ratio', 'Price to sales ratio', 'Price to cash flow ratio', 'Enterprise value to EBITDA ratio','Price to book ratio']
        title = 'Price Ratios'
        self.graph_template(y_axis_data=params, graph_title=title)

    def return_ratios(self):
        params = ['Return on assets %', 'Return on equity %', 'Return on invested capital %']
        title = 'Return Ratios'
        self.graph_template(y_axis_data=params, graph_title=title)

    def margins(self):
        params = ['Gross margin %', 'EBITDA margin %', 'Net margin %', 'Operating margin %']
        title = 'Margins'
        self.graph_template(y_axis_data=params, graph_title=title)

    def dept_ratios(self):
        params = ['Debt to assets ratio', 'Debt to equity ratio', 'Long term debt to total assets ratio']
        title = 'Dept Ratios'
        self.graph_template(y_axis_data=params, graph_title=title)

    def liquidity_ratios(self):
        params = ['Quick ratio', 'Current ratio', 'Inventory turnover', 'Asset turnover']
        title = 'Liquidity Ratios'
        self.graph_template(y_axis_data=params, graph_title=title)

    def graph_template(self, y_axis_data, graph_title):
            """
            y_axis_list=['Total revenue', 'Cost of goods sold', 'Gross profit'] 
            graph_title= f'{self.company_name} | Total Revenue - Cost Of Goods Sold = Gross Profit'
            """
            fig = px.line(self.df_statistics,
                            x = self.df_statistics.index, 
                            y = y_axis_data,
                            title = f'{self.company_name} | {graph_title}',
                            markers = True)

            fig.update_traces(
                mode="markers+lines", hovertemplate=None)
            fig.update_layout(
                hovermode="x", hoverlabel_namelength=-1)
            fig.show()


    """ OLD DEFs
    def __init__(self, company_data):

        self.company_name = company_data.company_data.loc['company_name'][0] #get company_name from the dataframe
        self.df_statistics = company_data.statistics.transpose()

    def shares_outstanding(self):
        fig_shares_outstanding = px.line(self.df_statistics, x=self.df_statistics.index, y=[
                                        'Total common shares outstanding', 'Float shares outstanding'], title='Number of Shares', markers=True)
        fig_shares_outstanding.show()
    
    def enterprice_values(self):

        fig_enterprice_values = px.line(self.df_statistics, x=self.df_statistics.index, y=[
                                        'Enterprise value'], title='Enterprise value', markers=True)
        fig_enterprice_values.show()
    
    def numer_of_employees_shareholders(self):

        fig_numer_of_employees_shareholders = px.line(self.df_statistics, x=self.df_statistics.index, y=[
                                                    'Number of employees', 'Number of shareholders'], 
                                                    title='Number of employees/shareholders', markers=True)
        fig_numer_of_employees_shareholders.show()

    def price_ratios(self):
        fig_price_ratios = px.line(self.df_statistics, x=self.df_statistics.index, y=['Price to earnings ratio', 'Price to sales ratio', 'Price to cash flow ratio', 'Enterprise value to EBITDA ratio',
                                                                            'Price to book ratio'], title='Price Ratios', markers=True)
        fig_price_ratios.show()

    def return_ratios(self):
        fig_return_ratios = px.line(self.df_statistics, x=self.df_statistics.index, y=['Return on assets %', 'Return on equity %', 'Return on invested capital %'],
                                    title='Return Ratios', markers=True)
        fig_return_ratios.show()

    def margins(self):
        fig_margins = px.line(self.df_statistics, x=self.df_statistics.index, y=[
                            'Gross margin %', 'EBITDA margin %', 'Net margin %', 'Operating margin %'], title='Margins', markers=True)
        fig_margins.show()

    def dept_ratios(self):
        fig_dept_ratios = px.line(self.df_statistics, x=self.df_statistics.index, y=[
                                'Debt to assets ratio', 'Debt to equity ratio', 'Long term debt to total assets ratio'], title='Dept Ratios', markers=True)
        fig_dept_ratios.show()

    def liquidity_ratios(self):
        fig_liquidity_ratios = px.line(self.df_statistics, x=self.df_statistics.index, y=[
                                    'Quick ratio', 'Current ratio', 'Inventory turnover', 'Asset turnover'], title='Liquidity Ratios', markers=True)
        fig_liquidity_ratios.show()
    """

In [11]:
#08 INPUT COMPANIES DATA CELL

companies = []
companies_urls = ['NASDAQ-AAPL', 'NASDAQ-TSLA', 'NYSE-BRK.A', 'NYSE-BRK.B', 'NYSE-KO', 'NYSE-BABA']

for url in companies_urls:

    #get the data from DataBase
    company_data = DataBase()
    company_data.GetFromDataBase(company_url=url)
    companies.append(company_data)

    #if data is not loaded from Database
    if company_data.income_statement is None or \
        company_data.balanse_sheet is None or \
        company_data.cashflow_statement is None or \
        company_data.statistics is None or \
        company_data.company_data is None:
        
        #get the data from the website
        company_data = ScrapeTrendingView(company_url=url)
        companies.append(company_data)

        #store the scraped data to DataBase
        DataBase.AddToDatabase(data=company_data)


#####################################################################
# companies = []
# company_urls = ['NASDAQ-AAPL', 'NASDAQ-TSLA', 'NYSE-BRK.A', 'NYSE-BRK.B']
# for url in company_urls:
#     company_data = ScrapeTrendingView(company_url=url)
#     companies.append(company_data)

# for company in companies:
#     if company.load_from_database == False:
#         DataBase.AddToDatabase(data=company)



#####################################################################
# apple = ScrapeTrendingView(company_url='NASDAQ-AAPL')
# apple.income_statement
# apple.balanse_sheet
# apple.cashflow_statement
# apple.statistics



INFO:root:Load NASDAQ-AAPL/Income-Statement from DataBase
INFO:root:Load NASDAQ-AAPL/Balance-Sheet from DataBase
INFO:root:Load NASDAQ-AAPL/Cashflow-Statement from DataBase
INFO:root:Load NASDAQ-AAPL/Ratios from DataBase
INFO:root:Load NASDAQ-AAPL/Company-Data from DataBase
INFO:root:Load NASDAQ-TSLA/Income-Statement from DataBase
INFO:root:Load NASDAQ-TSLA/Balance-Sheet from DataBase
INFO:root:Load NASDAQ-TSLA/Cashflow-Statement from DataBase
INFO:root:Load NASDAQ-TSLA/Ratios from DataBase
INFO:root:Load NASDAQ-TSLA/Company-Data from DataBase
INFO:root:Load NYSE-BRK.A/Income-Statement from DataBase
INFO:root:Load NYSE-BRK.A/Balance-Sheet from DataBase
INFO:root:Load NYSE-BRK.A/Cashflow-Statement from DataBase
INFO:root:Load NYSE-BRK.A/Ratios from DataBase
INFO:root:Load NYSE-BRK.A/Company-Data from DataBase
INFO:root:Load NYSE-BRK.B/Income-Statement from DataBase
INFO:root:Load NYSE-BRK.B/Balance-Sheet from DataBase
INFO:root:Load NYSE-BRK.B/Cashflow-Statement from DataBase
INFO:root:

In [12]:
#09 VISUALIZE ON SEPARATE CHARTS

for company in companies:
    fig = IncomeStatementVisualizer(company_data=company)
    # fig.show_all_visuals()
    # fig.revenue()
    # fig.operating_income()
    # fig.pretax_income()
    # fig.discontinued_operations()
    # fig.net_income()
    # fig.diluted_net_income()
    # fig.eps()
    # fig.shares()
    # fig.ebit()
    # fig.operating_expenses()

    fig = BalanceSheetVisualizer(company_data=company)
    # fig.show_all_visuals()
    # fig.book_value_per_share()

    fig = CashflowStatementVisualizer(company_data=company)
    # fig.show_all_visuals()
    # fig.cashflow_investing_activities()

    fig = StatisticsRatiosVisualizer(company_data=company)
    fig.show_all_visuals()
    break




In [44]:
#10 VISUALIZE ON COMBINED CHARTS

class CompareCompanies():

    def __init__(self, companies_data):
        self.companies_data = companies_data

    def income_statement(self, parameter_name):

        fig = go.Figure()

        for company in self.companies_data:

            columns = company.income_statement.columns
            rows = company.income_statement.loc[parameter_name]
            trace_name = f"{company.company_name}"# | {parameter_name}"
            fig = fig.add_trace(go.Scatter(x = columns, y= rows, name=trace_name))

        fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
        fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
        fig.update_layout(hovermode="x", hoverlabel_namelength=-1,  title=parameter_name) #display the full parameter name

        # fig.show()
                
        Excel.insert_visual(fig=fig, name=parameter_name)
        # Excel.insert_visual(fig=fig, cell='A2', name=parameter_name )
    
    def balanse_sheet(self, parameter_name):

        fig = go.Figure()

        for company in self.companies_data:

            columns = company.balanse_sheet.columns
            rows = company.balanse_sheet.loc[parameter_name]
            trace_name = f"{company.company_name}"# | {parameter_name}"
            fig = fig.add_trace(go.Scatter(x = columns, y= rows, name=trace_name))

        fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
        fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
        fig.update_layout(hovermode="x", hoverlabel_namelength=-1,  title=parameter_name) #display the full parameter name

        fig.show()

    def cashflow_statement(self, parameter_name):

        fig = go.Figure()

        for company in self.companies_data:

            columns = company.cashflow_statement.columns
            rows = company.cashflow_statement.loc[parameter_name]
            trace_name = f"{company.company_name}"# | {parameter_name}"
            fig = fig.add_trace(go.Scatter(x = columns, y= rows, name=trace_name))

        fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
        fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
        fig.update_layout(hovermode="x", hoverlabel_namelength=-1,  title=parameter_name) #display the full parameter name

        fig.show()

    def statistics_ratios(self, parameter_name):

        fig = go.Figure()

        for company in self.companies_data:

            columns = company.statistics.columns
            rows = company.statistics.loc[parameter_name]
            trace_name = f"{company.company_name}"# | {parameter_name}"
            fig = fig.add_trace(go.Scatter(x = columns, y= rows, name=trace_name))

        fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
        fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
        fig.update_layout(hovermode="x", hoverlabel_namelength=-1,  title=parameter_name) #display the full parameter name

        fig.show()

        

# temp2 = CompareCompanies_Statistics(companies_data=companies)
# temp2.total_comon_shares_outstanding()

Excel.start() #start Excel class

income_stat_params = IncomeStatementVisualizer(company_data=companies[0])
balanse_sh_params = BalanceSheetVisualizer(company_data=companies[0])
cashflow_params = CashflowStatementVisualizer(company_data=companies[0])
statistics_params = StatisticsRatiosVisualizer(company_data=companies[0])

# print(statistics_params.total_common_shares_outstanding_str)

compare = CompareCompanies(companies_data=companies)

#compare income statement
compare.income_statement(parameter_name=income_stat_params.total_revenue_str)
compare.income_statement(parameter_name=income_stat_params.cost_of_goods_sold_str)
compare.income_statement(parameter_name=income_stat_params.deprecation_and_amortization_str)
compare.income_statement(parameter_name=income_stat_params.depreciation_str)
compare.income_statement(parameter_name=income_stat_params.amortization_of_intangibles_str)
compare.income_statement(parameter_name=income_stat_params.amortization_of_deferred_charges_str)
compare.income_statement(parameter_name=income_stat_params.other_cost_of_goods_sold_str)
compare.income_statement(parameter_name=income_stat_params.gross_profit_str)
compare.income_statement(parameter_name=income_stat_params.operating_expenses_excl_cogs_str)
compare.income_statement(parameter_name=income_stat_params.selling_general_admin_expenses_total_str)
compare.income_statement(parameter_name=income_stat_params.research_and_development_str)
compare.income_statement(parameter_name=income_stat_params.selling_general_admin_expenses_other_str)
compare.income_statement(parameter_name=income_stat_params.other_operating_expenses_total_str)
compare.income_statement(parameter_name=income_stat_params.operating_income_str)
compare.income_statement(parameter_name=income_stat_params.non_operating_income_total_str)
compare.income_statement(parameter_name=income_stat_params.interest_expense_net_of_interest_capitalized_str)
compare.income_statement(parameter_name=income_stat_params.interest_expense_on_debt_str)
compare.income_statement(parameter_name=income_stat_params.interest_capitalized_str)
compare.income_statement(parameter_name=income_stat_params.non_operating_income_excl_interest_expenses_str)
compare.income_statement(parameter_name=income_stat_params.non_operating_interest_income_str)
compare.income_statement(parameter_name=income_stat_params.pretax_equity_in_earnings_str)
compare.income_statement(parameter_name=income_stat_params.miscellaneous_non_operating_expense_str)
compare.income_statement(parameter_name=income_stat_params.unusual_income_expense_str)
compare.income_statement(parameter_name=income_stat_params.impairments_str)
compare.income_statement(parameter_name=income_stat_params.restructuring_charge_str)
compare.income_statement(parameter_name=income_stat_params.legal_claim_expense_str)
compare.income_statement(parameter_name=income_stat_params.unrealized_gain_loss_str)
compare.income_statement(parameter_name=income_stat_params.other_exceptional_charges_str)
compare.income_statement(parameter_name=income_stat_params.pretax_income_str)
compare.income_statement(parameter_name=income_stat_params.equity_in_earnings_str)
compare.income_statement(parameter_name=income_stat_params.taxes_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_current_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_current_domestic_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_current_foreign_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_deferred_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_deferred_domestic_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_deferred_foreign_str)
compare.income_statement(parameter_name=income_stat_params.income_tax_credits_str)
compare.income_statement(parameter_name=income_stat_params.non_controlling_minority_interest_str)
compare.income_statement(parameter_name=income_stat_params.after_tax_other_income_expense_str)
compare.income_statement(parameter_name=income_stat_params.net_income_before_discontinued_operations_str)
compare.income_statement(parameter_name=income_stat_params.discontinued_operations_str)
compare.income_statement(parameter_name=income_stat_params.net_income_str)
compare.income_statement(parameter_name=income_stat_params.dilution_adjustment_str)
compare.income_statement(parameter_name=income_stat_params.preferred_dividends_str)
compare.income_statement(parameter_name=income_stat_params.diluted_net_income_available_to_common_stockholders_str)
compare.income_statement(parameter_name=income_stat_params.basic_earnings_per_share_basic_eps_str)
compare.income_statement(parameter_name=income_stat_params.diluted_earnings_per_share_diluted_eps_str)
compare.income_statement(parameter_name=income_stat_params.average_basic_shares_outstanding_str)
compare.income_statement(parameter_name=income_stat_params.diluted_shares_outstanding_str)
compare.income_statement(parameter_name=income_stat_params.ebitda_str)
compare.income_statement(parameter_name=income_stat_params.ebit_str)
compare.income_statement(parameter_name=income_stat_params.total_operating_expenses_str)



#compare statistics/ratios
# compare.statistics_ratios(parameter_name=statistics_params.total_common_shares_outstanding_str)
# compare.statistics_ratios(parameter_name=statistics_params.float_shares_outstanding_str)
# compare.statistics_ratios(parameter_name=statistics_params.number_of_employees_str)
# compare.statistics_ratios(parameter_name=statistics_params.number_of_shareholders_str)
# compare.statistics_ratios(parameter_name=statistics_params.price_to_earnings_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.price_to_sales_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.price_to_cash_flow_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.price_to_book_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.enterprise_value_str)
# compare.statistics_ratios(parameter_name=statistics_params.enterprise_value_to_ebitda_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.return_on_assets_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.return_on_equity_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.return_on_invested_capital_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.gross_margin_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.operating_margin_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.ebitda_margin_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.net_margin_percent_str)
# compare.statistics_ratios(parameter_name=statistics_params.quick_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.current_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.inventory_turnover_str)
# compare.statistics_ratios(parameter_name=statistics_params.asset_turnover_str)
# compare.statistics_ratios(parameter_name=statistics_params.debt_to_assets_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.debt_to_equity_ratio_str)
# compare.statistics_ratios(parameter_name=statistics_params.long_term_debt_to_total_assets_ratio_str)



""" 
    OLD CODE
    fig = go.Figure()

    for company in companies:
        i=0
        parameter_name = company.statistics.index[i]
        columns = company.statistics.columns
        rows = company.statistics.iloc[i]
        trace_name = f"{company.company_name}"# | {parameter_name}"
        fig = fig.add_trace(go.Scatter(x = columns, y= rows, name=trace_name))

    fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
    fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
    fig.update_layout(hovermode="x", hoverlabel_namelength=-1,  title=parameter_name) #display the full parameter name

    fig.show()
"""



INFO:root:A1
INFO:root:26
INFO:root:A26
INFO:root:51
INFO:root:A51
INFO:root:76
INFO:root:A76
INFO:root:101
INFO:root:A101
INFO:root:126
INFO:root:A126
INFO:root:151
INFO:root:A151
INFO:root:176
INFO:root:A176
INFO:root:201
INFO:root:A201
INFO:root:226
INFO:root:A226
INFO:root:251
INFO:root:A251
INFO:root:276
INFO:root:A276
INFO:root:301
INFO:root:A301
INFO:root:326
INFO:root:A326
INFO:root:351
INFO:root:A351
INFO:root:376
INFO:root:A376
INFO:root:401
INFO:root:A401
INFO:root:426
INFO:root:A426
INFO:root:451
INFO:root:A451
INFO:root:476
INFO:root:A476
INFO:root:501
INFO:root:A501
INFO:root:526
INFO:root:A526
INFO:root:551
INFO:root:A551
INFO:root:576
INFO:root:A576
INFO:root:601
INFO:root:A601
INFO:root:626
INFO:root:A626
INFO:root:651
INFO:root:A651
INFO:root:676
INFO:root:A676
INFO:root:701
INFO:root:A701
INFO:root:726
INFO:root:A726
INFO:root:751
INFO:root:A751
INFO:root:776
INFO:root:A776
INFO:root:801
INFO:root:A801
INFO:root:826
INFO:root:A826
INFO:root:851
INFO:root:A851
INFO:ro

' \n    OLD CODE\n    fig = go.Figure()\n\n    for company in companies:\n        i=0\n        parameter_name = company.statistics.index[i]\n        columns = company.statistics.columns\n        rows = company.statistics.iloc[i]\n        trace_name = f"{company.company_name}"# | {parameter_name}"\n        fig = fig.add_trace(go.Scatter(x = columns, y= rows, name=trace_name))\n\n    fig.update_xaxes(categoryorder=\'category ascending\')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)\n    fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse\n    fig.update_layout(hovermode="x", hoverlabel_namelength=-1,  title=parameter_name) #display the full parameter name\n\n    fig.show()\n'

In [ ]:
#comparison visials of two or more companies

# df_apple = apple.statistics.transpose()
# fig_apple = px.line(df_apple, x=df_apple.index, y=[
#                     'Number of employees'], title='Apple', markers=True)
# fig_apple.show()

# df_tesla = tesla.statistics.transpose()
# fig_tesla = px.line(df_tesla, x=df_tesla.index, y=[
#                     'Number of employees'], title='Tesla', markers=True)
# fig_tesla.show()

# apple.statistics.index

# fig = px.line(x=apple.statistics.columns, y=apple.statistics.loc['Number of employees'], title='', markers=True)
# fig.show()

# dict for the dataframes and their names

# plot the data
fig = go.Figure()

for company in companies:
    i=8
    company_name = company.company_name
    parameter_name = company.statistics.index[i]
    columns = company.statistics.columns
    rows = company.statistics.iloc[i]
    table_name = f"{company_name}/{parameter_name}"
    fig = fig.add_trace(go.Scatter(x = columns, y= rows, name= table_name))

fig.update_xaxes(categoryorder='category ascending')  # sort X-axis (when X-axis of different companies contains different ranges i.e. 2015-2021, 2016-2022)
fig.update_traces(mode="markers+lines", hovertemplate=None) #enable hover-mode, interactively display values on the graph when pointed with mouse
fig.update_layout(hovermode="x", hoverlabel_namelength=-1) #display the full parameter name

fig.show()


# fig =  px.line(x = tesla.statistics.columns, y = tesla.statistics.iloc[1])
# fig = px.line(x =apple.statistics.columns, y = apple.statistics.iloc[1])
# fig.show()

# apple.statistics.iloc[2]


In [ ]:
# plotting the data
plt.figure(figsize=(20, 8))  # set figure size


# tesla_data.financial_ratios_quarter.loc['Current Ratio'].plot()
# apple_data.financial_ratios_quarter.loc['Current Ratio'].plot()

# print(run_scraper.income_statement.index[0])
# param_name = run_scraper.income_statement.index[0] #get parameter name from dataframe columns

i = 0
companies[0].statistics.iloc[i].plot()
companies[1].statistics.iloc[i].plot()

# print(apple.statistics.iloc[i])
# print(tesla.statistics.iloc[i])
# run_scraper.income_statement.loc['Interest expense on debt'].plot()
# plt.plot()

# invert x-axis (2009 on the left, 2022 on the right)
# ax = plt.gca()
# ax.invert_xaxis()
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
    row=1, col=2
)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()